In [1]:
# 불러오기
import pandas as pd
df = pd.read_excel('2years.xlsx',skiprows = range(0,2))
df = df.loc[~(df['매체']=='패키지(표시포함)')]
# 참고사항 앞쪽만 split 하기
df['data'] = df['결과내용'].str.split('[').str[0]

# 특정원료
i_list = ['프로바이오틱스', '홍삼','EPA 및 DHA 함유 유지',
        '가르시니아캄보지아추출물','마리골드꽃추출물',  
        '밀크씨슬추출물' ,'프로폴리스추출물','디메틸설폰(MSM)']

idf = {}
for i in i_list:
    idf[i] = df[df['기능성원료']==i]
    
# final = pd.concat(x.values())



In [2]:
import re
from collections import Counter


def clean_text(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r'>(.*?)0', '', text)
    text = re.sub(r'>(.*?)\r\r', '', text)
    text = re.sub(r'0 ', '', text)
    text = re.sub(r'※ ', '', text)
    text = re.sub(r'\r', '', text)
    text = re.sub(r'/', '', text)
    text = re.sub(r'인체시험의 특정수치를(.*?)kg으로 변환할 것', '', text)
    text = re.sub(r'인체적용시험 결과를 인용하여 표현하는 경우(.*?)figure와 함께 인용됐을 경우 글자크기는 같거나 작게 광고할 것', '', text)              
    text = re.sub(r'인체적용시험 결과를 인용하여 표현하는 경우', '', text)
    text = re.sub(r' ', '', text)
    text = re.sub(r'인터넷(.*?)전화번호를 함께 광고할 것', '', text)
    text = re.sub(r'제조업소명(.*?)내용을포함할것', '', text)
    text = re.sub(r'건강정보와(.*?)명시할것', '', text)
    text = re.sub(r'1위', '', text)
    text = re.sub(r'19', '', text)
    text = re.sub(r'.루테인', '', text)
    text = re.sub(r'이미지', '', text)
    return text

fnctn = lambda x: clean_text(str(x))

In [3]:
data_clean = {}
data_strip = {}
result = {}
rdf = {}

for j in i_list:
    # apply function to dataframe
    data_clean[j] = pd.DataFrame(idf[j].data.apply(fnctn))
    # convert dataframe to long string
    data_strip[j] = data_clean[j]['data'].str.cat(sep= '')
    # split long string to big list
    data_strip[j] = re.split(r'-+', data_strip[j]) #sentdex
    # count most common words from the big list
    result[j] = Counter(data_strip[j]).most_common()
    # create dataframe with list
    rdf[j] = pd.DataFrame(result[j], columns=['수정삭제','개수'])
    # extract only top 20
    rdf[j] = rdf[j].iloc[0:20]
    rdf[j].drop(columns=['개수'], inplace = True)
    rdf[j].index = rdf[j].index + 1
    

In [4]:
fdf = pd.DataFrame()
fdf['프로바이오틱스'] = rdf['프로바이오틱스']['수정삭제']
fdf['홍삼'] = rdf['홍삼']['수정삭제']
fdf['EPA 및 DHA 함유 유지'] = rdf['EPA 및 DHA 함유 유지']['수정삭제']
fdf['가르시니아캄보지아추출물'] = rdf['가르시니아캄보지아추출물']['수정삭제']
fdf['마리골드꽃추출물'] = rdf['마리골드꽃추출물']['수정삭제']
# fdf['밀크씨슬추출물'] = rdf['밀크씨슬추출물']['수정삭제']
fdf['프로폴리스추출물'] = rdf['프로폴리스추출물']['수정삭제']
fdf['디메틸설폰(MSM)'] = rdf['디메틸설폰(MSM)']['수정삭제']
